In [1]:
import numpy as np
import pandas as pd
import os

In [2]:
import tensorflow as tf
import numpy as np
import pandas as pd 
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import numpy as np
from keras.utils import to_categorical
from sklearn import svm
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score
from sklearn.neighbors import KNeighborsClassifier
import datetime
tf.config.run_functions_eagerly(True)

##### **Loading, visualising the dataset, and preparing the data**

In [3]:
df_test = pd.read_csv (r'../input/fashionmnist/fashion-mnist_test.csv')
df_train = pd.read_csv (r'../input/fashionmnist/fashion-mnist_train.csv') 

In [4]:
a = df_train
train_f = a.drop('label',1)
b=df_test
test_features=b.drop('label',1)
train_l=a['label']
test_labels=b['label']
train_features, val_features, train_labels, val_labels = train_test_split(train_f, train_l, test_size=0.2, random_state=1)
print("shape of training features : "+ str(train_features.shape))
print("shape of training labels" + str(train_labels.shape))

print("shape of valuation features " + str(val_features.shape))
print("shape of valuation labels" + str(val_labels.shape))

print("shape of testing features" + str(test_features.shape))
print("shape of testing labels" + str(test_labels.shape))

In [5]:
#Transforming the Dataframes into arrays
train_features = train_features.to_numpy()
train_labels = train_labels.to_numpy()
val_features = val_features.to_numpy()
val_labels = val_labels.to_numpy()
test_features = test_features.to_numpy()
test_labels = test_labels.to_numpy()    

In [6]:
#Let's visualise some of the images that we are trying to classify
for i in range(25):
    plt.subplot(5,5,i+1)
    plt.xticks([])
    plt.yticks([])
    plt.imshow(train_features[i].reshape(28,28))
plt.show()   

In [7]:
plt.figure()
x1 = train_labels
plt.hist(x1, range = (0, 9), bins = 10, color = 'blue',
            edgecolor = 'black')
plt.xlabel('classes')
plt.ylabel('number')
plt.title('histogram for training labels')

plt.figure()
x2 = test_labels
plt.hist(x2, range = (0, 9), bins = 10, color = 'red',
            edgecolor = 'black')
plt.xlabel('classes')
plt.ylabel('number')
plt.title('histogram for testing labels')

**Scaling the data**

In [8]:
X=StandardScaler().fit_transform(train_features)
X_test=StandardScaler().fit_transform(test_features)

#### ***Support Vector Machine***

In [9]:
#We scale the training data and feed it to the SVC algorithm
model2=svm.SVC(kernel='poly',degree=2)
model2.fit(X,train_labels)

In [10]:
#We evaluate the accuracy of our model
model2.predict([X_test[0]])
y1 = test_labels
y2 = model2.predict(X_test)


In [11]:
print('the accuracy of SVM is : ' + str(accuracy_score(y1, y2, normalize=True)))

#### ***Convolution Neural network***

In [12]:
input_shape = (28, 28, 1) 


model5 = tf.keras.models.Sequential()
#the first convolution layer
model5.add(tf.keras.layers.Conv2D(32, kernel_size=(3, 3),
                 activation='relu',
                 kernel_initializer='he_normal',
                 input_shape=input_shape))
model5.add(tf.keras.layers.MaxPooling2D((2, 2)))


#the second convolution layer
model5.add(tf.keras.layers.Conv2D(64, (3, 3), activation='relu'))
model5.add(tf.keras.layers.MaxPooling2D(pool_size=(2, 2)))


#the third convolution layer
model5.add(tf.keras.layers.Conv2D(128, (3, 3), activation='relu'))


#classificator, a fully connected neural network that takes the outcome of the convolution as an input
model5.add(tf.keras.layers.Flatten())
model5.add(tf.keras.layers.Dense(128, activation='relu'))
model5.add(tf.keras.layers.Dropout(0.3))
model5.add(tf.keras.layers.Dense(10, activation='softmax'))

model5.compile(loss=tf.keras.losses.categorical_crossentropy,
              optimizer=tf.keras.optimizers.Adam(),
              metrics=['accuracy'])

model5.summary()

In [13]:
#reshaping the data for the convolution
train_featuresQ=train_features.reshape(train_features.shape[0],28,28,1)
test_featuresQ=test_features.reshape(test_features.shape[0],28,28,1)
val_featuresQ=val_features.reshape(val_features.shape[0],28,28,1)

train_featuresQ= train_featuresQ.astype('float32')
test_featuresQ = test_featuresQ.astype('float32')
val_featuresQ = val_featuresQ.astype('float32')
train_featuresQ /= 255
test_featuresQ /= 255
val_featuresQ /= 255

train_labelsQ=to_categorical(train_labels)
test_labelsQ=to_categorical(test_labels)
val_labelsQ=to_categorical(val_labels)

history = model5.fit(train_featuresQ,train_labelsQ,
                    batch_size=512,
                    epochs=15,
                    verbose=1,
                    validation_data=(val_featuresQ, val_labelsQ))




In [14]:
score = model5.evaluate(test_featuresQ, test_labelsQ, verbose=0)

In [15]:
print('the accuracy of CNN is : ' + str(score[1]))

> the accuracy of CNN is : 0.9185000061988831

In [16]:
%matplotlib inline
accuracy = history.history['accuracy']
val_accuracy = history.history['val_accuracy']
loss = history.history['loss']
val_loss = history.history['val_loss']
epochs = range(len(accuracy))
plt.plot(epochs, accuracy, 'bo', label='Training accuracy')
plt.plot(epochs, val_accuracy, 'b', label='Validation accuracy')
plt.title('Training and validation accuracy')
plt.legend()
plt.figure()
plt.plot(epochs, loss, 'bo', label='Training loss')
plt.plot(epochs, val_loss, 'b', label='Validation loss')
plt.title('Training and validation loss')
plt.legend()
plt.show()